In [1]:
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [2]:
groq_api_key=os.getenv("GROQ_API_KEY")
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001FD6C551A00>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001FD6C642750>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [12]:
model.invoke([HumanMessage(content="Hi , My name is Naresh and I am learning GenAI now")])

AIMessage(content="Hi Naresh,\n\nIt's great to meet you! \n\nLearning about Generative AI is exciting. It's a rapidly evolving field with incredible potential. \n\nWhat specifically are you interested in learning about GenAI? \n\nPerhaps I can help you with:\n\n* **Understanding the basics:** What is GenAI, how does it work, and what are some examples?\n* **Exploring different types of GenAI models:** Text generation, image generation, code generation, etc.\n* **Learning about specific applications:** Chatbots, creative writing, drug discovery, etc.\n* **Getting started with GenAI tools:** There are many open-source and commercial tools available.\n\nJust let me know what you'd like to explore, and I'll do my best to assist you.  Good luck with your learning journey!\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 176, 'prompt_tokens': 23, 'total_tokens': 199, 'completion_time': 0.32, 'prompt_time': 0.005006738, 'queue_time': 0.29292862, 'total_time': 

In [ ]:
model.invoke(
    [
        HumanMessage(content="Hi , My name is Naresh and I am learning GenAI now"),
        AIMessage(content="Hi Naresh,\n\nIt's great to meet you! \n\nLearning about Generative AI is exciting. It's a rapidly evolving field with incredible potential. \n\nWhat specifically are you interested in learning about GenAI? \n\nPerhaps I can help you with:\n\n* **Understanding the basics:** What is GenAI, how does it work, and what are some examples?\n* **Exploring different types of GenAI models:** Text generation, image generation, code generation, etc.\n* **Learning about specific applications:** Chatbots, creative writing, drug discovery, etc.\n* **Getting started with GenAI tools:** There are many open-source and commercial tools available.\n\nJust let me know what you'd like to explore, and I'll do my best to assist you.  Good luck with your learning journey!\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

# Here, models are stateless and it does not have previous context even if data is sent as history in messages

AIMessage(content="As an AI, I have no memory of past conversations and don't know your name or what you do. \n\nYou introduced yourself as Naresh at the beginning of our conversation.  \n\nIs there anything else I can help you with?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 219, 'total_tokens': 272, 'completion_time': 0.096363636, 'prompt_time': 0.010192032, 'queue_time': 0.24318567900000002, 'total_time': 0.106555668}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--e0a7fbe3-18ef-4239-92c4-eb7a11a53c4c-0', usage_metadata={'input_tokens': 219, 'output_tokens': 53, 'total_tokens': 272})

In [3]:
store={}
# Get seesion id which is stored in inmemory message history.
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

In [ ]:
with_message_history=RunnableWithMessageHistory(model,get_session_history)
config={"configurable":{"session_id":"chat1"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Naresh and I am learning GenAI now")],
    config=config
)
response.content

"Hi Naresh,\n\nIt's great to meet you!  I'm glad you're learning about Generative AI. It's a fascinating field with a lot of potential. \n\nWhat specifically are you interested in learning about GenAI?  \n\nDo you have any questions I can help you with, or are you just looking for information on where to start?  \n\nI can offer resources, explain concepts, or even generate some examples of GenAI in action. Just let me know!\n"

In [6]:
store

{'chat1': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi , My name is Naresh and I am learning GenAI now', additional_kwargs={}, response_metadata={}), AIMessage(content="Hi Naresh, it's great to meet you! \n\nWelcome to the world of Generative AI. It's a fascinating field with a lot of potential. What specifically are you interested in learning about GenAI? \n\nPerhaps I can help you with:\n\n* **Understanding the basics:** What is GenAI, its capabilities, and how it works?\n* **Exploring different types of GenAI models:** Text generation, image generation, code generation, etc.\n* **Learning about specific applications:** Chatbots, creative writing, drug discovery, etc.\n* **Getting hands-on with GenAI tools:** There are many open-source and commercial tools available.\n\nJust let me know what you'd like to explore, and I'll do my best to guide you.  Good luck with your learning journey!\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 

In [7]:
type(store['chat1'])

langchain_core.chat_history.InMemoryChatMessageHistory

In [15]:
# Create chat prompt template
prompt=ChatPromptTemplate(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)
parser=StrOutputParser()
chain=prompt|model|parser

In [16]:
chain.invoke({'messages': [HumanMessage(content="Define GenAI in 10 words")]})

'Generative Artificial Intelligence: AI that creates new content.  \n'

In [18]:
# Run with runnable message history to run along with previous session history messages
with_message_history=RunnableWithMessageHistory(chain,get_session_history)
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Naresh")],
    config=config
)
response

"Hello Naresh, it's nice to meet you!\n\nI'm happy to help with any questions you have. Just ask away, and I'll do my best to provide you with useful and informative answers.  😊 \n\nWhat can I do for you today? \n\n"

In [4]:
# Adding params to system or human messages
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        (
            "human",
            "Explain about {topic} with in 5 words.",
        ),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | model
# chain.invoke({'messages':[HumanMessage(content='Elaborate the topic adding 2 extra words')], 'language':'English','topic':'GenAI'})

In [27]:
prompt.invoke({'messages':[HumanMessage(content='Elaborate the topic adding 2 extra words')], 'language':'English','topic':'GenAI'})

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant. Answer all questions to the best of your ability in English.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Explain about GenAI with in 5 words.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Elaborate the topic adding 2 extra words', additional_kwargs={}, response_metadata={})])

In [ ]:
"""
Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history."""

with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "chat5"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Elaborate the topic adding 2 extra words")],"language":"Hindi", 'topic':'Machine learning'},
    config=config
)
repsonse.content

'मशीनों को सीखने में सहायता करना \n'

In [41]:
store['chat5']

InMemoryChatMessageHistory(messages=[HumanMessage(content='Elaborate the topic adding 2 extra words', additional_kwargs={}, response_metadata={}), AIMessage(content='जनरेटिव आर्टिफिशियल इंटेलिजेंस  \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 56, 'total_tokens': 76, 'completion_time': 0.036363636, 'prompt_time': 0.003478754, 'queue_time': 0.243668316, 'total_time': 0.03984239}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--14c2334d-217a-463b-b460-3c96fe26f054-0', usage_metadata={'input_tokens': 56, 'output_tokens': 20, 'total_tokens': 76}), HumanMessage(content='Elaborate the topic adding 2 extra words', additional_kwargs={}, response_metadata={}), AIMessage(content='मशीनों को सीखने में सहायता करना \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 93, 'total_tokens': 110, 'completion_time': 0.0309090

### Trimming the Conversation History

In [5]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

e:\Development\langchain\langchain_models\lang_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
e:\Development\langchain\langchain_models\lang_venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\User\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingf

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [19]:
prompt2 = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain2 = prompt2 | model

In [30]:
# filter messages through chain
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
chain2=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt2
    | model
)
response=chain2.invoke(
    {
    "messages":[HumanMessage(content="what math problem did i ask")],
    "language":"English"
    }
)
response.content

"As a large language model, I have no memory of past conversations. To help me answer your question, please provide the math problem you'd like me to solve. 😊  \n\n"

In [37]:
chain2=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt2
    | model
)
chain2.invoke(
    {
    "messages":messages + [HumanMessage(content="what math problem did i ask")],
    "language":"English"
    }
)

AIMessage(content='You asked "what\'s 2 + 2". 😊  \n\n\n\nDo you want to try another one?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 86, 'total_tokens': 112, 'completion_time': 0.047272727, 'prompt_time': 0.005314469, 'queue_time': 0.24331381000000002, 'total_time': 0.052587196}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--164273a5-9df2-4c9f-a360-2ead0febbced-0', usage_metadata={'input_tokens': 86, 'output_tokens': 26, 'total_tokens': 112})

In [38]:
# Send filtered messages through RunnableWithMessageHistory
with_message_history = RunnableWithMessageHistory(
    chain2,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat7"}}

In [39]:
response = with_message_history.invoke(
    {
        "messages":messages + [HumanMessage(content="what math problem did i ask?")],
        "language": "English"
    },
    config=config,
)

response.content

'You asked "what is 2 + 2" 😊  \n\n\n\n'

In [40]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_core.messages import trim_messages
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from operator import itemgetter
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv

# Load API key
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")

# Initialize model
model = ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)

# Define trimmer
trimmer = trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

# Message history
history = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

# Define prompt
prompt2 = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="messages"),
    ("human", "{input}")
])

# Chain with correct trimming and history injection
chain2 = (
    RunnablePassthrough.assign(
        messages=RunnableLambda(itemgetter("messages")) | trimmer
    )
    | prompt2
    | model
)

# Invoke with current input
response = chain2.invoke({
    "messages": history,
    "input": "what math problem did I ask?"
})

# Print the output
print(response.content)


You asked "whats 2 + 2"  🍦  



Let me know if you'd like to try another one! 😊


## Second & correct type of chat history runnable

In [57]:
# Get seesion id which is stored in inmemory message history.
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory() # ChatMessageHistory save Humanmessage and AImessages
    return store[session_id]
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Respond only in {language}."),
    MessagesPlaceholder(variable_name="messages"),
    ("human","{input}")
])
parser=StrOutputParser()
base_chain = prompt | model | parser
with_message_history=RunnableWithMessageHistory(
    base_chain,
    get_session_history,
    input_messages_key="input", # Current input
    history_messages_key="messages" # This gets and saves old and current input dfrom user 
)
config = {"configurable": {"session_id": "chat11"}}

response = with_message_history.invoke(
    {
        "input": "Explain about ML in 5 words",
        "language":"English"
    },
    config=config
)

print(response)

Data-driven decision making by machines. 

